# Wine Types Analysis Using Deep Learning Exercise Solution

Adapted from Dipanjan Sarkar et al. 2018. [Practical Machine Learning with Python](https://link.springer.com/book/10.1007/978-1-4842-3207-1).

## Overview

This module focuses on building predictive models to predict wine types (red or white wine) based on various features using Deep Learning techniques. The analysis uses a dataset containing wine quality measurements and characteristics.

## Learning Objectives

- Build and train a Deep Neural Network model for wine type classification
- Understand feature scaling and preprocessing techniques
- Evaluate model performance using various metrics
- Visualize model training progress and results
- Apply binary classification techniques using deep learning

### Tasks to complete

- Build and compile a Deep Neural Network model
- Train the model using scaled features
- Evaluate model performance on test data
- Generate performance visualizations and metrics

## Prerequisites

- Python programming environment
- Basic understanding of statistical and machine learning concepts
- Familiarity with common ML libraries

## Get Started

- Please select kernel "conda_tensorflow2_p310" from SageMaker notebook instance.


In [ ]:
# Import necessary dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
from keras.models import Sequential
from keras.layers import Dense
from numpy import interp

%matplotlib inline

## Load and merge datasets


In [ ]:
# Load datasets
red_wine = pd.read_csv("../../Data/winequality-red.csv", sep=";")
white_wine = pd.read_csv("../../Data/winequality-white.csv", sep=";")

# Add wine type and quality labels
red_wine["wine_type"] = "red"
white_wine["wine_type"] = "white"

def categorize_quality(value):
    """Categorize wine quality into low, medium, or high."""
    if value <= 5:
        return "low"
    elif value <= 7:
        return "medium"
    else:
        return "high"

red_wine["quality_label"] = red_wine["quality"].apply(categorize_quality)
white_wine["quality_label"] = white_wine["quality"].apply(categorize_quality)

# Merge datasets and shuffle
wines = pd.concat([red_wine, white_wine]).sample(frac=1, random_state=42).reset_index(drop=True)

# Display dataset info
print("Dataset Info:")
print(wines.info())
print("\nSample Data:")
print(wines.head())


### Understand dataset features and values


In [ ]:
print(white_wine.shape, red_wine.shape)
print(wines.info())

We have 4898 white wine data points and 1599 red wine data points. The
merged dataset contains a total of 6497 data points and we also get an idea of numeric and categorical
attributes.


In [ ]:
# Let’s take a peek at our dataset to see some sample data points.
wines.head()

## Utilty functions for model evaluation


In [ ]:
# Define helper functions for model evaluation
def get_metrics(true_labels, predicted_labels):
    """Calculate and print performance metrics."""
    print("Accuracy:", np.round(metrics.accuracy_score(true_labels, predicted_labels), 4))
    print("Precision:", np.round(metrics.precision_score(true_labels, predicted_labels, average="weighted"), 4))
    print("Recall:", np.round(metrics.recall_score(true_labels, predicted_labels, average="weighted"), 4))
    print("F1 Score:", np.round(metrics.f1_score(true_labels, predicted_labels, average="weighted"), 4))

def display_classification_report(true_labels, predicted_labels, classes):
    """Display classification report."""
    print(metrics.classification_report(true_labels, predicted_labels, labels=classes))

def display_confusion_matrix(true_labels, predicted_labels, classes):
    """Display confusion matrix."""
    cm = metrics.confusion_matrix(true_labels, predicted_labels, labels=classes)
    cm_frame = pd.DataFrame(cm, index=classes, columns=classes)
    print("Confusion Matrix:")
    print(cm_frame)

def display_model_performance_metrics(true_labels, predicted_labels, classes):
    """Display model performance metrics."""
    print("Model Performance Metrics:")
    print("-" * 30)
    get_metrics(true_labels, predicted_labels)
    print("\nClassification Report:")
    print("-" * 30)
    display_classification_report(true_labels, predicted_labels, classes)
    print("\nConfusion Matrix:")
    print("-" * 30)
    display_confusion_matrix(true_labels, predicted_labels, classes)

def plot_model_roc_curve(clf, features, true_labels, label_encoder=None, class_names=None):
    """Plot ROC curve for the model."""
    if hasattr(clf, "classes_"):
        class_labels = clf.classes_
    elif label_encoder:
        class_labels = label_encoder.classes_
    elif class_names:
        class_labels = class_names
    else:
        raise ValueError("Unable to derive prediction classes!")

    n_classes = len(class_labels)
    y_test = label_binarize(true_labels, classes=class_labels)

    if n_classes == 2:
        y_score = clf.predict_proba(features)[:, 1] if hasattr(clf, "predict_proba") else clf.decision_function(features)
        fpr, tpr, _ = roc_curve(y_test, y_score)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"ROC curve (area = {roc_auc:.2f})", linewidth=2.5)
    else:
        y_score = clf.predict_proba(features) if hasattr(clf, "predict_proba") else clf.decision_function(features)
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test[:, i], y_score[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f"ROC curve of class {class_labels[i]} (area = {roc_auc:.2f})", linestyle=":", linewidth=2)

    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.show()

## Predicting Wine Types

We will predict the wine type based on other features. To start with, we
will first select our necessary features and separate out the prediction class labels and prepare train and test
datasets. We use the prefix **wtp\_** in our variables to easily identify them as needed, where **wtp** depicts wine
type prediction.


In [ ]:
wtp_features = wines.iloc[:, :-3]
wtp_feature_names = wtp_features.columns
wtp_class_labels = np.array(wines["wine_type"])

# prepare train and test datasets
wtp_train_X, wtp_test_X, wtp_train_y, wtp_test_y = train_test_split(
    wtp_features, wtp_class_labels, test_size=0.2, random_state=42
)

print(Counter(wtp_train_y), Counter(wtp_test_y))
print("Features:", list(wtp_feature_names))

The numbers show us the wine samples for each class and we can also see the feature names which will
be used in our feature set.


### Feature Scaling

We will be using a standard scaler in this scenario.


In [ ]:
# Scale features
scaler = StandardScaler().fit(wtp_train_X)
wtp_train_SX = scaler.transform(wtp_train_X)
wtp_test_SX = scaler.transform(wtp_test_X)

### Train a Model using Deep Learning (MLP)

Let’s try
modeling the data using a fully connected deep neural network (DNN) with three hidden layers.


#### Encodes wine type class labels


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
wtp_train_ey = label_encoder.fit_transform(wtp_train_y)
wtp_test_ey = label_encoder.transform(wtp_test_y)

In [ ]:
print(wtp_train_SX.shape)
print(wtp_train_ey.shape)
print(wtp_train_SX.nbytes / 1024**2)  # Size in MB

#### Build & Compile DNN Model Architecture

Let’s build the architecture for our three-hidden layer DNN where each hidden layer has 16 units (the
input layer has 11 units for the 11 features) and the output layer has 1 unit to predict a 0 or 1, which maps
back to red or white wine.


In [ ]:
# Build a sequential model of three-hidden layer DNN where each hidden layer has 16 units and 'relu' activation
# (the input layer has 11 units for the 11 features) and the output layer has 1 unit and 'sigmoid' activation
# Build and train the DNN model

# Your code goes here

# Configures the model for training use 'Adam' as optimizer, 'binary_crossentropy' 
# as loss funciton, and 'accuracy' as evaluation metrics

# Your code goes here


#### Train the Model

We use 10% of the training data for a validation set while training the model to see how it performs at
each epoch


In [ ]:
history = wtp_dnn_model.fit(wtp_train_SX, wtp_train_ey, epochs=10, batch_size=5, shuffle=True, validation_split=0.1, verbose=1)

#### Predict on test data

Let’s now predict and evaluate our model on the actual test dataset.


In [ ]:
wtp_dnn_ypred = wtp_dnn_model.predict(wtp_test_SX)
wtp_dnn_ypred = (wtp_dnn_ypred > 0.5).astype(int)  # More efficient thresholding
wtp_dnn_predictions = le.inverse_transform(np.ravel(wtp_dnn_ypred))

#### Evaluate Model Performance


In [ ]:
display_model_performance_metrics(
    true_labels=wtp_test_y,
    predicted_labels=wtp_dnn_predictions,
    classes=["red", "white"],
)

We get an overall F1 Score and model accuracy of 99.5%, which is even
better than our previous model! This goes to prove you don’t always need big data but good quality data
and features even for Deep Learning models.


In [ ]:
# Plot the loss and accurcy measures at each epoch
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
t = f.suptitle("Deep Neural Net Performance", fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epochs = list(range(1, 11))
ax1.plot(epochs, history.history["accuracy"], label="Train Accuracy")
ax1.plot(epochs, history.history["val_accuracy"], label="Validation Accuracy")
ax1.set_xticks(epochs)
ax1.set_ylabel("Accuracy Value")
ax1.set_xlabel("Epoch")
ax1.set_title("Accuracy")
l1 = ax1.legend(loc="best")

ax2.plot(epochs, history.history["loss"], label="Train Loss")
ax2.plot(epochs, history.history["val_loss"], label="Validation Loss")
ax2.set_xticks(epochs)
ax2.set_ylabel("Loss Value")
ax2.set_xlabel("Epoch")
ax2.set_title("Loss")
l2 = ax2.legend(loc="best")

## Conclusion

Through this module, we successfully implemented a complete deep learning workflow including:

- Data preprocessing and feature scaling techniques
- Building a three-layer Deep Neural Network architecture
- Training the model with validation monitoring
- Implementing comprehensive model evaluation metrics
- Creating performance visualizations to track model training
- Applying binary classification techniques to real-world data

These practical skills demonstrate the end-to-end process of developing and deploying deep learning models for classification tasks.

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
